# LABELS

In [3]:
import os
import re
UASpeechMLF_FilesPath=r'E:\UAspeesh dataset\mlf'
UASpeechMLF_FilesName=[file for file in os.listdir(UASpeechMLF_FilesPath)if file.endswith('.mlf')]

## Convert MLF Files To Multi TXT Files (107100) in Folder (files)

In [6]:
if not os.path.isdir('files'):
    os.mkdir('files')
ErrorFileNameInLineDict={}
for mlf_File in UASpeechMLF_FilesName: 
    with open(UASpeechMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
        mlf_FileBufferWords=mlf_FileBuffer.readlines()
    CounterOfmlf_FileBufferWords=0
    for line in mlf_FileBufferWords:
        CounterOfmlf_FileBufferWords+=1
        if re.match(r'"\*/\w+.lab"',line):
            try:
                with open('files/'+re.sub('.lab"\n','',re.sub(r'"\*/','',line))+'.txt','w') as WordFile:
                    WordFile.write(mlf_FileBufferWords[CounterOfmlf_FileBufferWords])
            except:
                ErrorFileNameInLineDict[mlf_File+line]=CounterOfmlf_FileBufferWords


## Print If There Erorr OR Not 

In [8]:
print(ErrorFileNameInLineDict)

{}


## Convert MLF Files To one Dict {FileName:Word}

In [4]:
allmlf_FilesWordsInDict={}
for mlf_File in UASpeechMLF_FilesName: 
    with open(UASpeechMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
        mlf_FileBufferWords=mlf_FileBuffer.readlines()
    CounterOfmlf_FileBufferWords=0
    for line in mlf_FileBufferWords:
        CounterOfmlf_FileBufferWords+=1
        if re.match(r'"\*/\w+.lab"',line):
            allmlf_FilesWordsInDict[re.sub('"\n','',re.sub(r'"\*/','',line))]=mlf_FileBufferWords[CounterOfmlf_FileBufferWords]

## convert all mlf files to one txt file as Dict (file:word)

In [7]:
if not os.path.isdir('files'):
    os.mkdir('files')
with open('files/labels.txt','w') as newLabelFile:
        for mlf_file in allmlf_FilesWordsInDict:
            newLabelFile.writelines(mlf_file+':'+allmlf_FilesWordsInDict[mlf_file])

## convert Values Of Dict Files to lists

In [ ]:
print(len(allmlf_FilesWordsInDict))
wordsInmlf_FilesInList=list(allmlf_FilesWordsInDict.values())
audioFilesNameInList=list(allmlf_FilesWordsInDict.keys())

# AUDIO

In [26]:
import os
UASpeechMainFilesOfDataPath=r'E:\UAspeesh dataset\audio' #edit path
UASpeechMainFilesOfDataName=[folder for folder in os.listdir(UASpeechMainFilesOfDataPath)if os.path.isdir(UASpeechMainFilesOfDataPath+'\\'+folder)] #to get only Folders

In [27]:
print(UASpeechMainFilesOfDataName)

['CF02', 'CF03', 'CF04', 'CM04', 'CM05', 'CM06', 'CM08', 'CM09', 'CM10', 'CM12', 'CM13', 'contro', 'F02', 'F03', 'F04', 'M01', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16']


In [32]:
UASpeechDataFilesPaths=[]
for mainFile in UASpeechMainFilesOfDataName:#main Files as CF02
    for mainFileChiled in os.listdir(UASpeechMainFilesOfDataPath+'\\'+mainFile):#git records in every main file 
        if not os.path.isdir(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled):#check if it file or Folder(control)
            print(mainFileChiled)
            if mainFileChiled.endswith('.wav'):
                UASpeechDataFilesPaths.append(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled)#append Path of file .wav 
        else:
            for mainFileLeaf in os.listdir(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled): #if main folder have folder then data 
                if not os.path.isdir(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled+'\\'+mainFileLeaf):#check if it file or another Folder              
                    if mainFileLeaf.endswith('.wav'):
                        UASpeechDataFilesPaths.append(UASpeechMainFilesOfDataPath+'\\'+mainFile+'\\'+mainFileChiled+'\\'+mainFileLeaf)

CF02_word.mlf
F1_S1_0001.wav
index.html
index.html@C=D;O=A
index.html@C=D;O=D
index.html@C=M;O=A
index.html@C=M;O=D
index.html@C=N;O=A
index.html@C=N;O=D
index.html@C=S;O=A
index.html@C=S;O=D
CF03_word.mlf
index.html
index.html@C=D;O=A
index.html@C=D;O=D
index.html@C=M;O=A
index.html@C=M;O=D
index.html@C=N;O=A
index.html@C=N;O=D
index.html@C=S;O=A
index.html@C=S;O=D
CF04_word.mlf
index.html
index.html@C=D;O=A
index.html@C=D;O=D
index.html@C=M;O=A
index.html@C=M;O=D
index.html@C=N;O=A
index.html@C=N;O=D
index.html@C=S;O=A
index.html@C=S;O=D
CM04_word.mlf
index.html
index.html@C=D;O=A
index.html@C=D;O=D
index.html@C=M;O=A
index.html@C=M;O=D
index.html@C=N;O=A
index.html@C=N;O=D
index.html@C=S;O=A
index.html@C=S;O=D
CM05_word.mlf
index.html
index.html@C=D;O=A
index.html@C=D;O=D
index.html@C=M;O=A
index.html@C=M;O=D
index.html@C=N;O=A
index.html@C=N;O=D
index.html@C=S;O=A
index.html@C=S;O=D
CM06_word.mlf
index.html
index.html@C=D;O=A
index.html@C=D;O=D
index.html@C=M;O=A
index.html@C=M;O=D

In [33]:
UASpeechDataFilesPaths

['E:\\UAspeesh dataset\\mlf\\CF02\\F1_S1_0001.wav',
 'E:\\UAspeesh dataset\\mlf\\contro\\CF02\\F1_S1_0001.wav']

# Extract features form waves